In [37]:
from tensorflow.keras.layers import Embedding,Input,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk import FreqDist
from string import punctuation as pt

In [22]:

# 텍스트 데이터 가져오기.
train_df=pd.read_table('./data/ratings_train.txt')
test_df=pd.read_table('./data/ratings_test.txt')

In [23]:
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [24]:
train_df.document = train_df.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex=True)  #한글을 제외한 단어는 정규식으로 제거

In [25]:
train_df['document'].isna().sum()   # 결측치의 수 확인

5

In [26]:
train_df.dropna(inplace=True)   #결측치 제거

In [27]:
# 데이터셋 사전 관련 상수
NUM_SIZE=10000
OOV='<UNK>'

tokenizer=Tokenizer(num_words=NUM_SIZE,oov_token=OOV)

In [28]:
# sentences 데이터 기반 단어 사전 생성
tokenizer.fit_on_texts(train_df['document'])

In [29]:
tokenizer.word_counts, tokenizer.word_index

(OrderedDict([('아', 1664),
              ('더빙', 126),
              ('진짜', 6213),
              ('짜증나네요', 17),
              ('목소리', 108),
              ('흠포스터보고', 1),
              ('초딩영화줄오버연기조차', 1),
              ('가볍지', 17),
              ('않구나', 2),
              ('너무재밓었다그래서보는것을추천한다', 1),
              ('교도소', 4),
              ('이야기구먼', 1),
              ('솔직히', 943),
              ('재미는', 292),
              ('없다평점', 1),
              ('조정', 11),
              ('사이몬페그의', 1),
              ('익살스런', 2),
              ('연기가', 743),
              ('돋보였던', 23),
              ('영화스파이더맨에서', 1),
              ('늙어보이기만', 1),
              ('했던', 138),
              ('커스틴', 4),
              ('던스트가', 1),
              ('너무나도', 193),
              ('이뻐보였다', 1),
              ('막', 192),
              ('걸음마', 1),
              ('뗀', 2),
              ('세부터', 1),
              ('초등학교', 52),
              ('학년생인', 1),
              ('살용영화ㅋㅋㅋ별반개도', 1),
              ('아까움', 252),
             

In [63]:
stopwords = ['너무','다','적','진짜','에서','점','영화','정말','의','가',
                    '이','은','들','는','좀','잘','걍','과','도','를','으로','자',
                    '에','와','한','하다','것','그','안','인','이런','내','못','게',
                    '이다','고','더','때','거','요','본','지','네','입니다','있는',
                    '걸','이렇게','임','라','인데','듯','기','서','난','면','대','라고',
                    '보면','이나','때문','해서','된','엔','두','이고','랑','전','애',
                    '없이','해','속','한다','라는','뿐','되는','니','줄','냐','하게',
                    '하고','합니다','번','하지','보고','이건','그냥','왜','ㅋㅋ','이거',
                    '이게','그리고','게다가','수','영화를','영화가','영화는','참','아','역시',
                    '다시','나옴','또','꼭','하는','없고','ㅋㅋㅋ','ㅠㅠ','어제','봤는데','없는','없다',
                    '보는','많이','뭐','무슨','ㅋ','암튼','ㅎ','ㅎㅎ','ㅎㅎㅎ','ㅡㅡ','내가','볼','대한','만든','벌써']

In [64]:
#토큰화


# KoNLPy의 Okt를 사용하여 토큰화
from tqdm import tqdm

# 현재 텍스트를 분리 => 공백기준
my_words=[]
for st in tqdm(train_df.document):
    ret=st.split()
    for _ in ret:
        if _ not in stopwords and _ not in pt:
            my_words.append(_)

100%|██████████| 149995/149995 [00:01<00:00, 119234.95it/s]


In [65]:
# 토큰화 데이터를 기반, 단어집합을 생성.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(my_words)

In [66]:
# 전체 단어 갯수
word_index=len(tokenizer.word_index)
word_index

299477

In [67]:
# 생성 단어집합 출력.
tokenizer.word_index

{'최고의': 1,
 '드라마': 2,
 '완전': 3,
 '평점': 4,
 '좋은': 5,
 '평점이': 6,
 '연기': 7,
 '쓰레기': 8,
 '최고': 9,
 '스토리': 10,
 '재밌게': 11,
 '가장': 12,
 '아깝다': 13,
 '마지막': 14,
 '영화다': 15,
 '같은': 16,
 '하지만': 17,
 '작품': 18,
 '별로': 19,
 '끝까지': 20,
 '솔직히': 21,
 '넘': 22,
 '봐도': 23,
 '전혀': 24,
 '말이': 25,
 '영화의': 26,
 '아주': 27,
 '내내': 28,
 '같다': 29,
 '지금': 30,
 '최악의': 31,
 '좋다': 32,
 '시간': 33,
 '할': 34,
 '뭔가': 35,
 '다른': 36,
 '재미': 37,
 '있다': 38,
 '그래도': 39,
 '않고': 40,
 '재미있게': 41,
 '어떻게': 42,
 '많은': 43,
 '중': 44,
 '하나': 45,
 '연기가': 46,
 '별': 47,
 '감독': 48,
 '재밌다': 49,
 '굿': 50,
 '이걸': 51,
 '아니라': 52,
 '이야기': 53,
 '연기도': 54,
 '모두': 55,
 '봤다': 56,
 '아니다': 57,
 '근데': 58,
 '아름다운': 59,
 '때문에': 60,
 '스토리가': 61,
 '보다가': 62,
 '명작': 63,
 '나는': 64,
 '감동': 65,
 '계속': 66,
 '제대로': 67,
 '나오는': 68,
 '영화에': 69,
 '느낌': 70,
 '제일': 71,
 '내용도': 72,
 '모든': 73,
 '내용이': 74,
 '시간이': 75,
 '점도': 76,
 '않은': 77,
 '보다': 78,
 '차라리': 79,
 '아직도': 80,
 '액션': 81,
 '아니고': 82,
 '재미없다': 83,
 '아닌': 84,
 '감독이': 85,
 '년': 86,
 '감독의': 87,
 '스토리도': 88,
 '

In [69]:
vocab = FreqDist(np.hstack(my_words))    # 빈도수를 알아서 구해주는 함수와 클래스들
vocab_size = 500                          # top 500만 사용하기
vocab = vocab.most_common(vocab_size)     
print(f'단어 집합의 크기 : {len(vocab)}')
print('빈도수 상위 10선:',vocab[:10])

단어 집합의 크기 : 500
빈도수 상위 10선: [('최고의', 2231), ('드라마', 1861), ('완전', 1814), ('평점', 1805), ('좋은', 1745), ('평점이', 1614), ('연기', 1596), ('쓰레기', 1503), ('최고', 1451), ('스토리', 1412)]


In [70]:
vocab

[('최고의', 2231),
 ('드라마', 1861),
 ('완전', 1814),
 ('평점', 1805),
 ('좋은', 1745),
 ('평점이', 1614),
 ('연기', 1596),
 ('쓰레기', 1503),
 ('최고', 1451),
 ('스토리', 1412),
 ('재밌게', 1347),
 ('가장', 1184),
 ('아깝다', 1172),
 ('마지막', 1090),
 ('영화다', 1066),
 ('같은', 1025),
 ('하지만', 970),
 ('작품', 948),
 ('별로', 946),
 ('끝까지', 946),
 ('솔직히', 943),
 ('넘', 911),
 ('봐도', 911),
 ('전혀', 882),
 ('말이', 870),
 ('영화의', 852),
 ('아주', 844),
 ('내내', 842),
 ('같다', 834),
 ('지금', 827),
 ('최악의', 817),
 ('좋다', 814),
 ('시간', 806),
 ('할', 805),
 ('뭔가', 805),
 ('다른', 797),
 ('재미', 785),
 ('있다', 775),
 ('그래도', 768),
 ('않고', 765),
 ('재미있게', 753),
 ('어떻게', 752),
 ('많은', 747),
 ('중', 746),
 ('하나', 745),
 ('연기가', 743),
 ('별', 735),
 ('감독', 713),
 ('재밌다', 712),
 ('굿', 711),
 ('이걸', 709),
 ('아니라', 698),
 ('이야기', 693),
 ('연기도', 692),
 ('모두', 689),
 ('봤다', 687),
 ('아니다', 686),
 ('근데', 683),
 ('아름다운', 678),
 ('때문에', 675),
 ('스토리가', 672),
 ('보다가', 672),
 ('명작', 666),
 ('나는', 663),
 ('감동', 660),
 ('계속', 657),
 ('제대로', 653),
 ('나오는', 650),
 ('영화

In [34]:
# 문장 ==> 숫자로 변환
# sentences=['Today is a sunny day', 
        #    'Today is a rainy day',
        #    'Is it sunny today?']
rets=tokenizer.texts_to_sequences(train_df.document)
rets

[[38, 908, 5, 6633, 1078],
 [1, 1, 6634, 1],
 [1],
 [1, 1, 75, 363, 1, 9721],
 [1, 1, 106, 4979, 1, 1, 824, 1, 1, 560, 1],
 [565, 1, 1, 1, 2246, 1, 1, 424],
 [1059, 2103, 128, 1],
 [107, 1205, 60, 3389, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [715, 1, 96, 34, 5204, 2],
 [1185, 39, 1, 282, 3211, 8993, 1, 3, 1, 967],
 [1],
 [685, 1, 1, 1, 1, 1],
 [2138, 1, 1, 1, 1, 5956, 42, 1],
 [9722, 1, 90, 1, 139, 58, 512, 1, 1, 1, 276],
 [3390, 1, 5, 1875, 237, 21, 285, 59, 452, 1, 150, 1, 312, 4980],
 [1, 2452, 1, 1],
 [37, 373, 9723, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1118],
 [7864, 1, 1, 1, 7, 1305, 255, 1],
 [27, 4, 1, 8994, 1, 1, 1, 15, 1, 1],
 [1, 243, 1, 5957, 1, 1, 189, 1],
 [169, 3950, 1, 34, 108, 8, 1, 1, 1, 18, 189, 337],
 [159, 8373, 152, 238, 1],
 [145,
  175,
  69,
  5958,
  3951,
  1,
  1,
  79,
  1,
  1,
  3509,
  235,
  5958,
  1,
  435,
  27,
  1,
  1,
  73,
  3391,
  1,
  1,
  1,
  1571,
  73,
  2925,
  67],
 [189, 728, 19, 146, 1, 1],
 [2986, 331, 2987, 261],
 [14, 1, 263, 1, 11, 1, 1, 57, 1],
 [1,

In [35]:
# 전체 문장의 길이 통일 ==> 패딩
lengths=[len(x) for x in rets]
TOKEN_LENTH=max(lengths)